<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#CatBoost" data-toc-modified-id="CatBoost-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>CatBoost</a></span></li><li><span><a href="#Тестирование" data-toc-modified-id="Тестирование-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Тестирование</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Машинное обучение для текстов

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Цель: обучить модель классифицировать комментарии на позитивные и негативные. Имеется в распоряжении набор данных с разметкой о токсичности правок. Необходимо построить модель со значением метрики качества *F1* не меньше 0.75. 

**План**

1. Загрузить и подготовьте данные.
2. Обучить разные модели. 
3. Сделайть выводы.


**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [ ]:
#!pip install pymystem3
#!python.exe -m pip install --upgrade pip
#!pip install torch torchvision torchaudio -f https://download.pytorch.org/whl/cu115/torch_stable.html

In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.utils.class_weight import compute_class_weight

from tqdm import tqdm
from tqdm.notebook import tqdm
from tqdm import notebook
tqdm.pandas()

import torch

import transformers
from transformers import BertTokenizer
from transformers import AutoTokenizer
from transformers import BertConfig

from catboost import CatBoostClassifier

import warnings
warnings.filterwarnings("ignore")

In [2]:
print(torch.cuda.is_available())

False


In [3]:
try:
    data = pd.read_csv('/datasets/toxic_comments.csv')
#    data = pd.read_csv('toxic_comments.csv')
except:
    print('no data')
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [5]:
print('Количество дубликатов в данных', data.duplicated().sum())

Количество дубликатов в данных 0


In [6]:
# выводим количество пропущенных значений каждого стобца
data.isna().sum()

Unnamed: 0    0
text          0
toxic         0
dtype: int64

Возььмем 1000 sample для ускорения расчетов.

In [7]:
data = data.sample(1000).reset_index(drop=True)

Вычислим веса классов в выбранном датасете:

In [8]:
class_weights = compute_class_weight(
    "balanced", classes=np.unique(data["toxic"]), y=data["toxic"])

print("Веса классов: ", class_weights)


Веса классов:  [0.56433409 4.38596491]


In [9]:
tokenizer = AutoTokenizer.from_pretrained("unitary/toxic-bert")

# Ограничим максимальную длину последовательности до 512 токенов
max_len = 512
tokenized = data["text"].apply(
    (
        lambda x: tokenizer.encode(
            x, add_special_tokens=True, max_length=max_len, truncation=True
        )
    )
)

padded = np.array([i + [0] * (max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

In [10]:
config = BertConfig.from_pretrained("unitary/toxic-bert")
model = transformers.BertModel.from_pretrained('unitary/toxic-bert', config=config)

Some weights of the model checkpoint at unitary/toxic-bert were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Эмбеддинги модель BERT создаёт батчами. Чтобы хватило оперативной памяти, сделаем размер батча небольшим:
- Сделаем цикл по батчам. Отображать прогресс будет функция notebook():

In [11]:
batch_size = 5
embeddings = []

# Переместить данные на устройство CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
padded = torch.LongTensor(padded).to(device)
attention_mask = torch.LongTensor(attention_mask).to(device)
model.to(device)

for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
    batch = padded[batch_size * i : batch_size * (i + 1)]
    attention_mask_batch = attention_mask[batch_size * i : batch_size * (i + 1)]

    with torch.no_grad():
        batch_embeddings = model(batch, attention_mask=attention_mask_batch)

    embeddings.append(batch_embeddings[0][:, 0, :].cpu().numpy())

  0%|          | 0/200 [00:00<?, ?it/s]

Разобьем данные на обучающую, валидационную и тестовые выборки:

In [15]:
features = np.concatenate(embeddings)
target = data['toxic']

#features_train, feautures_valid, target_train, target_valid  = train_test_split\
#(features, target, test_size=0.4, random_state=256, stratify = target)

#features_valid, features_test, target_valid, target_test  = train_test_split\
#(feautures_valid, target_valid, test_size=0.5, random_state=256, stratify = target_test)  

features_train, features_test, target_train, target_test  = train_test_split\
(features, target, test_size=0.2, random_state=256, stratify = target)

print('Размер тренировочной выборки:', features_train.shape[0]/data.shape[0]*100, '%')
#print('Размер валидационной выборки:', features_valid.shape[0]/data.shape[0]*100, '%' )
print('Размер тестовой выборки:', features_test.shape[0]/data.shape[0]*100, '%')

Размер тренировочной выборки: 80.0 %
Размер тестовой выборки: 20.0 %


## Обучение

### Логистическая регрессия

In [ ]:
#model = LogisticRegression(random_state=256, solver='lbfgs',class_weight='balanced')
#model.fit(features_train, target_train)
#predicted_valid = model.predict(features_valid)
#print('F1-score:', f1_score(target_valid, predicted_valid))

F1-score: 0.5909090909090908

In [17]:
# определение сетки параметров для перебора
param_grid = {'C': [0.1, 1, 10, 100], 'penalty': ['l1', 'l2']}

# создание модели
log_reg = LogisticRegression(random_state=256, class_weight='balanced')
scorer = make_scorer(f1_score)
# создание объекта GridSearchCV и запуск перебора параметров
grid_search = GridSearchCV(log_reg, param_grid=param_grid, cv=5, scoring=scorer)
grid_search.fit(features_train, target_train)

# вывод наилучших параметров и оценки f1-score 
print(f'Лучшие параметры: {grid_search.best_params_}')
print(f'Лучшая оценка F1-score: {grid_search.best_score_:.3f}') 
#print('CV results:', grid_search.cv_results_['mean_test_score'])

Лучшие параметры: {'C': 10, 'penalty': 'l2'}
Лучшая оценка F1-score: 0.939


Лучшие параметры: {'C': 10, 'penalty': 'l2'}
Лучшая оценка F1-score: 0.939

### CatBoost

In [ ]:
#%%time
#model = CatBoostClassifier(class_weights=dict(enumerate(class_weights)),eval_metric='F1')
#model.fit(features_train, target_train, verbose=500)
#predicted_valid = model.predict(features_valid)
#print('F1-score:', f1_score(target_valid, predicted_valid))

Подберем гиперпараметры для CatBoost

In [18]:
# Задаем сетку гиперпараметров
param_grid = {
    'iterations': [10, 50, 100],
    'learning_rate': [0.01, 0.1, 0.5],
    'depth': [3, 5, 7],
    'l2_leaf_reg': [1, 3, 5]
}

In [19]:
model = CatBoostClassifier(class_weights=dict(enumerate(class_weights)))

grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=5,
    scoring='f1',
    n_jobs=-1
)

In [20]:
%%time
# Обучаем модель с использованием GridSearchCV
grid_search.fit(features_train, target_train)

# Выводим лучшие гиперпараметры и оценку f1_score
print(f'Лучшие параметры: {grid_search.best_params_}')
print(f'Лучшая оценка F1-score: {grid_search.best_score_:.3f}') 
print('CV results:', grid_search.cv_results_['mean_test_score'])

0:	learn: 0.5379220	total: 160ms	remaining: 7.85s
1:	learn: 0.3992281	total: 180ms	remaining: 4.32s
2:	learn: 0.3117042	total: 200ms	remaining: 3.13s
3:	learn: 0.2564816	total: 220ms	remaining: 2.52s
4:	learn: 0.2084423	total: 241ms	remaining: 2.17s
5:	learn: 0.1790456	total: 263ms	remaining: 1.93s
6:	learn: 0.1531090	total: 284ms	remaining: 1.74s
7:	learn: 0.1348920	total: 304ms	remaining: 1.6s
8:	learn: 0.1211194	total: 324ms	remaining: 1.48s
9:	learn: 0.0974672	total: 345ms	remaining: 1.38s
10:	learn: 0.0881559	total: 366ms	remaining: 1.3s
11:	learn: 0.0841876	total: 390ms	remaining: 1.24s
12:	learn: 0.0790907	total: 415ms	remaining: 1.18s
13:	learn: 0.0758945	total: 439ms	remaining: 1.13s
14:	learn: 0.0679346	total: 462ms	remaining: 1.08s
15:	learn: 0.0659190	total: 486ms	remaining: 1.03s
16:	learn: 0.0611006	total: 508ms	remaining: 987ms
17:	learn: 0.0571121	total: 531ms	remaining: 944ms
18:	learn: 0.0563245	total: 554ms	remaining: 904ms
19:	learn: 0.0531428	total: 576ms	remaining

Лучшие параметры: {'depth': 3, 'iterations': 50, 'l2_leaf_reg': 1, 'learning_rate': 0.1}
Лучшая оценка F1-score: 0.920

### Тестирование

Лучше всего показала себя логистичекая регрессия с параметрами: Лучшие параметры: {'C': 10, 'penalty': 'l2'}
Лучшая оценка F1-score: 0.939

In [21]:
model = LogisticRegression(random_state=256, C= 10, penalty= 'l2', class_weight='balanced')
model.fit(features_train, target_train)
predicted_test = model.predict(features_test)
print('F1-score:', f1_score(target_test, predicted_test))

F1-score: 0.92


## Выводы

F1-score: 0.92 на тестовой выборке была получена с помощью модели линейной регрессии.Был взят сэмпл из 1000 выборок из всего датасета и получены признаки с помощью предобученной модели БЕРТ на определение токсичности.